In [2]:
%load_ext autoreload
%autoreload 2

import sys
from os import path

root_path = path.abspath(path.join('..', '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

# `Logit` on Orders - Logistic Regression (~1h)

## Select features

🎯 Haydi `wait_time` ve `delay_vs_expected` değişkenlerinin çok `iyi/kötü review`lar üzerindeki etkisini inceleyelim.

👉 `orders` training_set’imizi kullanarak iki adet `multivariate logistic regression` çalıştıracağız:
- `logit_one` → `dim_is_one_star` tahmini için  
- `logit_five` → `dim_is_five_star` tahmini için.

 

In [3]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Dataset’inizi import edin:

In [4]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Kullanmak istediğiniz feature’ları bir listede seçin:

⚠️ Data leakage yaratmadığınızdan emin olun (yani target’tan türetilmiş feature’ları seçmeyin)

💡 `wait_time` ve `delay_vs_expected` değişkenlerinin etkisini anlayabilmek için diğer feature’ların etkisini kontrol etmemiz gerekir, bu yüzden listenize ilgili olabilecek tüm feature’ları dahil edin.

In [5]:
orders.head()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_items,number_of_sellers,price,freight_value,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72,18.063837
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76,856.292580
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22,514.130333
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20,1822.800366
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72,30.174037


In [6]:
data_logit = orders.copy()

In [7]:
features = [
    'wait_time',
    'delay_vs_expected',
    'number_of_items',
    'number_of_sellers',
    'price',
    'freight_value',
    'distance_seller_customer'
]

In [8]:
data_logit['dim_is_one_star'] = (data_logit['review_score'] == 1).astype(int)
data_logit['dim_is_five_star'] = (data_logit['review_score'] == 5).astype(int)

🕵🏻 Feature’larınızın `multicollinearity` durumunu `VIF index` kullanarak kontrol edin.

* Çok yüksek olmamalıdır (tercihen < 10), böylece partial regression coefficient’larına ve ilgili `p-values` değerlerine güvenebiliriz.
* Verinizi standardize etmeyi unutmayın!
    * Bir `VIF Analysis`, bir feature’ın diğer feature’lara karşı regresyonunu yaparak hesaplanır...
    * Bu yüzden herhangi bir linear regression çalıştırmadan önce feature’ların `scale etkisini kaldırmak` ve eşit öneme sahip olmalarını sağlamak istersiniz!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardize etme:

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

data_logit_std = data_logit.copy()

data_logit_std[features] = scaler.fit_transform(data_logit[features])

👉 Olası multicollinearity durumlarını analiz etmek için VIF Analysis’inizi çalıştırın:

In [16]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = data_logit_std[features]

vif_data = pd.DataFrame()
vif_data["feature"] = features
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(features))]

print(vif_data.sort_values(by="VIF", ascending=False))

                    feature       VIF
0                 wait_time  2.624944
1         delay_vs_expected  2.213500
5             freight_value  1.673727
6  distance_seller_customer  1.442040
2           number_of_items  1.371316
4                     price  1.208582
3         number_of_sellers  1.093349


## Logistic Regressions

👉 İki adet `Logistic Regression` modeli fit edin:
- `logit_one` → `dim_is_one_star` tahmini için
- `logit_five` → `dim_is_five_star` tahmini için.

`Logit 1️⃣`

In [17]:
formula = "dim_is_one_star ~ " + " + ".join(features)

logit_one = smf.logit(formula=formula, data=data_logit_std).fit()

print(logit_one.summary())

Optimization terminated successfully.
         Current function value: 0.273582
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:        dim_is_one_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Fri, 02 Jan 2026   Pseudo R-squ.:                  0.1448
Time:                        15:31:34   Log-Likelihood:                -26229.
converged:                       True   LL-Null:                       -30669.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.4676      0.013   -190.852      0.000      -2.493

`Logit 5️⃣`

In [18]:
formula_five = "dim_is_five_star ~ " + " + ".join(features)

logit_five = smf.logit(formula=formula_five, data=data_logit_std).fit()

print(logit_five.summary())

Optimization terminated successfully.
         Current function value: 0.636830
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:       dim_is_five_star   No. Observations:                95872
Model:                          Logit   Df Residuals:                    95864
Method:                           MLE   Df Model:                            7
Date:                Fri, 02 Jan 2026   Pseudo R-squ.:                 0.05806
Time:                        15:32:12   Log-Likelihood:                -61054.
converged:                       True   LL-Null:                       -64817.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.3388      0.007     47.340      0.000       0.325

💡 Şimdi bu iki logistic regression’ın sonuçlarını analiz etme zamanı:

- Partial coefficient’ları kendi kelimelerinizle yorumlayın.
- `p-values` kullanarak istatistiksel anlamlılıklarını kontrol edin.
- Coefficient önemleri açısından `logit_one` ve `logit_five` arasında herhangi bir fark görüyor musunuz?

wait_time: Her iki modelde de en baskın katsayıdır. 1-yıldız modelinde pozitif ($0.71$), 5-yıldız modelinde negatiftir ($-0.52$). Yani bekleme süresi, bir müşteriyi mutsuz etmedeki en güçlü silahtır.

delay_vs_expected: Bekleme süresi aynı kalsa bile, müşteriye verilen sözün tutulmaması (gecikme) 1-yıldız olasılığını artırırken, 5-yıldız alma şansını ciddi oranda düşürür.

distance_seller_customer: İlginç bir şekilde, mesafe arttıkça 1-yıldız verme olasılığı düşüyor ($-0.17$). Bu, müşterilerin uzak mesafeden gelen ürünler için daha sabırlı ve anlayışlı olma eğiliminde olduğunu gösterir.

Çoğu değişken için p-değeri 0.000'dır. Bu, bu değişkenlerin müşteri puanını etkilemediği yönündeki "Null Hypothesis"i reddettiğimiz ve sonuçların tesadüf olmadığını, istatistiksel olarak yüksek derecede anlamlı olduğunu kanıtlar.

Ancak 5-yıldız modelinde freight_value için p-değeri 0.555 çıkmıştır. Bu, nakliye ücretinin bir müşterinin 5 yıldız verip vermemesi üzerinde anlamlı bir etkisi olmadığını gösterir.

wait_time, 1-yıldız verme kararında ($0.71$) 5-yıldız verme kararından ($|-0.52|$) daha etkilidir. Yani yavaşlık, müşteriyi kaybetmekte, hızlılık ise müşteriyi kazanmakta olduğundan daha güçlüdür.

delay_vs_expected, 5-yıldız modelinde ($|-0.43|$) 1-yıldız modeline ($0.25$) göre çok daha büyük bir katsayıya sahiptir. Bu, "mükemmel bir deneyim" yaşatmak istiyorsanız asla gecikmemeniz gerektiğini, çünkü gecikmenin 5 yıldızı anında yok ettiğini gösterir.

In [19]:
# Aşağıdaki cümlelerden doğru olanları aşağıdaki listeye kaydedin.

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more than one_star"

your_answer = [a]

🧪 __Kodunu Test Et__

In [20]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/ilos/.pyenv/versions/3.12.9/envs/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/ilos/code/inurekici.sc/projects/S14D3Ps--olist-analysis/02-Regression-Models/data-logit/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_logit.py::TestLogit::test_question PASSED                           [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/logit.pickle

git commit -m 'Completed logit step'

git push origin master



<details>
    <summary>- <i>Açıklamalar ve ileri seviye kavramlar</i> -</summary>


> _Diğer tüm şeyler sabitken, `delay factor`, 1-yıldız review alma ihtimalini etkilemesinden bile daha fazla, 5-yıldızdan mahrum kalma ihtimalini artırma eğilimindedir. Muhtemelen bunun sebebi, 1-yıldız review’ların bizzat çok kötü ürünleri hedeflemesi, kötü teslimatları değil._

❗️ Ancak tamamen titiz olmak için, **iki farklı modelin coefficient’larını karşılaştırırken daha dikkatli olmamız gerekir**, çünkü **benzer popülasyonlara dayanmayabilirler**!
    Burada 2 alt popülasyonumuz var: (1-yıldız verenler ve 5-yıldız verenler) ve bunlar doğaları gereği farklı davranış kalıpları sergileyebilirler. 5-yıldız vermeye daha meyilli “mutlu insanlar”ın, “gecikme” veya “fiyat” söz konusu olduğunda, 1-yıldızı “Lucky-Luke gibi ateşleyen” “huysuz insanlara” göre daha az hassas olmaları gayet mümkün...

</details>



🏁 Tebrikler!

💾 `logit.ipynb` notebook’unuzu commit ve push etmeyi unutmayın!